# Wilcoxon Test for Accuracy

In [4]:
from numpy.random import seed
from numpy.random import randn
from scipy.stats import wilcoxon
import pandas as pd
import itertools

In [87]:
# Wilcoxon signed-rank test

class MyWilcoxon:
    def __init__(self, sheet_path=r'C:\Users\HassanEldeeb\Documents\GitHub\AutoMLBenchmarking\Complete_Sheet.xlsx',
                sheet_names=['10 Min', '30 Min', '60 Min', '4 Hours', 'meta-features']):
        
        
        self.sheet = pd.read_excel(sheet_path,
                              null_values=['', 'NA', 'NAN', 'NaN', 'Nan', 'NA\n','0', '0.0', 'None'],
                              sheet_name=sheet_names)
        
        
        
    def calc_wilcoxon(self):
        for t in ['10 Min', '30 Min', '60 Min', '4 Hours']:
            self.sheet[t].rename(columns={"sklearn_e_accuracy_mean": "sklearn-e_accuracy_mean",
                                            "sklearn_m_accuracy_mean": "sklearn-m_accuracy_mean",
                                            "sklearn_v_accuracy_mean": "sklearn-v_accuracy_mean"}, inplace=True)
        result = pd.DataFrame(columns = ['Factor_1', 'Factor_2', 'Time_Budget_1', 'Time_Budget_2',
                                         'p_value', 'stat', 'Statistically_Better'])
        cols = ['sklearn_accuracy_mean', 'sklearn-e_accuracy_mean', 'sklearn-m_accuracy_mean', 'sklearn-v_accuracy_mean',
                'autoweka_accuracy_mean', 'recipe_valid_acc', 'smartml_valid_acc', 'tpot_accuracy_mean', 'atm_acc']
        for t in itertools.product(['10 Min', '30 Min', '60 Min', '4 Hours'], ['10 Min', '30 Min', '60 Min', '4 Hours']):
            for col in itertools.product(cols, cols):
                if (t[0]== t[1] and col[0] != col[1]) or ((t[0]!= t[1] and col[0] == col[1])):
                    data1 = self.sheet[t[0]][col[0]]
                    data2 = self.sheet[t[1]][col[1]]
                    condition = (data1 != 0) & (data2 != 0)
                    data1 = data1[condition]
                    data2 = data2[condition]
                    
                    stat, p = wilcoxon(data1, data2)
                    if p <= 0.05:
                        if data1.mean() > data2.mean():
                            better = '1'
                        elif data2.mean() > data1.mean():
                            better = '2'
                        else:
                            better = 'None'
                    else:
                        better = 'None'
                    new_row = {'Factor_1': col[0].split('_')[0], 'Factor_2': col[1].split('_')[0], 'Time_Budget_1':t[0], 'Time_Budget_2':t[1],
                                'p_value': p, 'stat': stat, 'Statistically_Better': better}
                    result = result.append(new_row, ignore_index=True)
        return result
            


In [88]:
wil = MyWilcoxon()

In [89]:
result = wil.calc_wilcoxon()

In [90]:
result[((result.Statistically_Better == '1') ^ (result.Statistically_Better == 'None')) & (((result.Factor_1=='sklearn-v') & (result.Factor_2=='sklearn-m')) ^ ((result.Factor_2=='sklearn-v') & (result.Factor_1=='sklearn-m')))].round(3)
#result[(result.Statistically_Better == '1') & ((result.Factor_1=='sklearn-v') ^ (result.Factor_2=='sklearn-v'))].round(3)

,Factor_1,Factor_2,Time_Budget_1,Time_Budget_2,p_value,stat,Statistically_Better
18,sklearn-m,sklearn-v,10 Min,10 Min,0.004,967.5,1
126,sklearn-m,sklearn-v,30 Min,30 Min,0.065,1076.0,None
134,sklearn-v,sklearn-m,30 Min,30 Min,0.065,1076.0,None
234,sklearn-m,sklearn-v,60 Min,60 Min,0.434,1689.0,None
242,sklearn-v,sklearn-m,60 Min,60 Min,0.434,1689.0,None
342,sklearn-m,sklearn-v,4 Hours,4 Hours,0.305,1406.0,None
350,sklearn-v,sklearn-m,4 Hours,4 Hours,0.305,1406.0,None
